# 2. 데이터 탐색과 전처리

이전 장에서는 Question Generation Task의 개념과 모델, 평가지표 등에 대해 알아보았습니다. 이번 장에서는 실습에 사용할 데이터셋을 확인해보도록 하겠습니다. 

NLP 튜토리얼에서는 에세이 데이터를 활용해 학습자들을 위한 영어 문제를 생성하는 모델을 만들어 볼 것입니다. 구체적으로 3개의 문제를 만들어 볼 것입니다. 왜 에세이냐? -> 개인의 생각이 들어간 수필인데 이것을 쓰면 이러이러한 이유에서 좋다  NUS는 세계 몇 위 대학이고, 학생들 수준이 상당히 높아 에세이 수준도 높다. RC로 적합하다 또 WI Locness는 상위 수준의 학생들의 에세이만 고를 꺼기 때문에, 괜찮다. 

문제를 모두 4지선다 문제다. 
왜 4지선다 문제냐? 평가 쉽고, blah blah blah

지문 내용 기반 참 거짓 문제, WH-5 문제 생성할거다. 

참 거짓 문제란 ~~

WH-5 문제란 ~~ 

grammar 문제는 실제 GEC task에 사용되는 데이터셋에서 틀린 grammar들 위주로 구성했으므로, 실제 ESL 학생드리 헷갈려하는 문제 위주로 grammar 문제를 구성해서 유효성을 확보함. 

2.1절에서는 데이터를 다운로드 받는 방법을 살펴보겠습니다. 2.2절에서는 데이터를 시각화 해볼 것이며, 2.3절에서는 matplotlib.pyplot 에서 제공하는 subplots 함수를 활용해 여러 이미지를 하나의 출력 창에 시각화 하는 방법을 알아보겠습니다. 마지막으로 2.4절에서는 이미지 전처리를 통해 픽셀값에 정규화(normalization)을 진행해보겠습니다.



## 2.1 2014 CoNLL Shared Task Dataset

Conference on Computational Natural Language Learning (CoNLL)은 매년 자연어처리 관련 기술에 대해 논하는 학회입니다. 논문 발표와 더불어 매년 Shared Task를 운영하고 있습니다. Shared Task는 CoNLL에서 운영하는 자연어 처리 경진대회입니다. CoNLL에서 운영한 모든 Shared Task는 [https://conll.org/previous-tasks](https://conll.org/previous-tasks)에서 확인 가능합니다. 

2013년, 2014년에는 Grammer Error Correction(GEC) 과제를 Shared Task로 운영했습니다. GEC는 에세이내의 문법 오류 위치를 탐지한 후 올바른 단어로 수정하는 알고리즘을 구축하는 과제였습니다.

이번 튜토리얼에서 사용할 데이터는 CoNLL Shared Task가 종료된 후 주최측에서 공개한 라벨링된 시험 데이터셋 입니다. Shared Task에서 공식적으로 사용된 훈련 데이터인 NUCLE corpus 같은 경우 [NUS Natural Language Processing Group](https://www.comp.nus.edu.sg/~nlp/corpora.html)에 신청서를 보내야 얻을 수 있지만, 시험 데이터셋은 누구나 열람할 수 있게 공개가 되어 있기 때문입니다. 

2014년도 GEC의 시험 데이터셋은 총 50개의 에세이로 이뤄져 있습니다. 25명의 비영어권 NUS 학생 (Non-native speaker of English) 이 그림 XX에 있는 2개의 주제에 대해 각각 하나의 에세이를 제출해서 총 50개 입니다. 해당 에세이에 대해 2명의 영어 원어민이 문법 교정을 실시했습니다. 



- Creator: National University of Singapore
- Year: 2013, 2014
- Purpose: 
    - shared task of the Seventeenth Conference on Computational Natural Language Learning
    - the
goal is to detect and correct the grammatical errors
present in the essay, and return the corrected essay (grammatical error correction)
- Data:
    - Written by ESL students, corrected by Error annotation on the test essays was carried
out by a native speaker of English who is a lecturer
at the NUS Centre for English Language Communication

    - 50 essays (1,312 sentences) on 2 different topics written by 25 South-East Asian undergraduates [Ref. BEA-2019 paper]

- Papers: https://www.aclweb.org/anthology/W13-3601.pdf

### 2.1.1 CoNLL 데이터 탐색

먼저 아래 코드를 활용해 https://www.comp.nus.edu.sg/~nlp/conll14st.html 에 공개된 시험 데이터셋을 Colab환경에 다운로드 받습니다. 리눅스의 `wget`명령어를 활용해 제공된 다운로드 링크로 부터 `tar.gz`파일을 받을 수 있습니다. 이때 `-O` 옵션을 주어 파일명을 `conll2014.tar.gz`로 변경해서 다운로드 받아보도록 하겠습니다. 

In [ ]:
!wget -O 'conll2014.tar.gz' https://www.comp.nus.edu.sg/~nlp/conll14st/conll14st-test-data.tar.gz

--2021-04-04 04:49:52--  https://www.comp.nus.edu.sg/~nlp/conll14st/conll14st-test-data.tar.gz
Resolving www.comp.nus.edu.sg (www.comp.nus.edu.sg)... 45.60.31.225
Connecting to www.comp.nus.edu.sg (www.comp.nus.edu.sg)|45.60.31.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 643482 (628K) [application/x-gzip]
Saving to: ‘conll2014.tar.gz’

conll2014.tar.gz    100%[===================>] 628.40K   448KB/s    in 1.4s    

2021-04-04 04:49:55 (448 KB/s) - ‘conll2014.tar.gz’ saved [643482/643482]



Colab 파일 경로에 그림 XX와 같이 `conll2014.tar.gz`파일이 다운로드 된것을 확인할 수 있습니다. 다음으로는 해당 파일을 리눅스의 `tar`명령어를 활용해 압축 해제 하겠습니다. 

In [ ]:
# 압축 풀고
!tar -xf conll2014.tar.gz

압축이 해제되면 `conll14st-test-data`폴더가 생성된 것을 확인할 수 있습니다. 해당 폴더는 그림 XX와 같이 구성돼 있습니다. 

그림 XX

각 폴더가 의미하는 내용은 다음과 같습니다. 

- `alt`: 참가팀에서 제출한 대체 답안
- `noalt`: 대체 답안 없이 주최측의 답안으로만 구성된 문서
- `scripts`: 주최측에서 제공한

scripts는 파이썬 2.6.4에서 원할히 작동합니다. 하지만 최근에는 대부분 파이썬 3으로 교육을 하기 때문에 튜토리얼에 scripts파일로 튜토리얼을 구성하기 적합하지 않습니다. 또한, scripts에 있는 코드를 배워 CoNLL Shared Task에 한정된 코드를 배워보는 것 보다는, 일반적으로 사용되는 BeautifulSoup, Numpy 등의 패키지로 직접 전처리 하는 과정을 연습해보면 좋을 것 같다는 판단이 들어, 본 튜토리얼에서는 scripts에서 제공한 파일을 쓰지 않고, 자체적으로 전처리 과정을 구성해보겠습니다. 전처리 하는 과정을 직접 익혀봄으로써 다른 데이터셋이 주어졌을 때 유동적으로 대처하는 능력이 배양되길 바랍니다. 

먼저 `BeautifulSoup` 라이브러리를 불러옵니다. 

In [ ]:
from bs4 import BeautifulSoup

`noalt`폴더 내에 있는 `official-2014.0.sgml`파일을 `open()` 함수를 활용해 엽니다.

In [ ]:
# 파일 열고 annotator 1에 의해 라벨링 된 파일
sgml_file = open('conll14st-test-data/noalt/official-2014.0.sgml')

그 후 `read()` 함수를 활용해 `official-2014.0.sgml` 파일의 내용물을 읽어서 `conll` 변수에 저장합니다. 

In [ ]:
# 파일 읽고
conll = sgml_file.read()
sgml_file.close()
type(conll)

str

`conll` 변수 타입을 확인해보니 문자열로 저장된 것을 확인할 수 있습니다. `conll` 변수에 어떤 문자열이 저장됐는지 확인해보겠습니다. 

In [ ]:
conll[:2500]

'<DOC nid="1">\n<TEXT>\n<TITLE>\nKeeping the Secret of Genetic Testing\n</TITLE>\n<P>\nWhat is genetic risk? Genetic risk refers more to your chance of inheriting a disorder or disease. People get certain disease because of genetic changes. How much a genetic change tells us about your chance of developing a disorder is not always clear. If your genetic results indicate that you have gene changes associated with an increased risk of heart disease, it does not mean that you definitely will develop heart disease. The opposite is also true. If your genetic results show that you do not have changes associated with an increased risk of heart disease, it is still possible that you develop heart disease. However for some rare diseases, people who have certain gene changes are guaranteed to develop the disease. When we are diagonosed out with certain genetic disease, are we suppose to disclose this result to our relatives? My answer is no.\n</P>\n<P>\nOn one hand, we do not want this potential

보시다 시피 `conll`변수 내에 저장된 문자열에 다양한 태그(tag)들이 존재하는 것을 확인할 수 있습니다. `<DOC>`, `<TEXT>`, `<TITLE>` 등 의 태그들이 존재합니다. 이것은 Markup Language를 표현하는 방식과 같습니다. 사실 파일 확장자 명에 있는 `sgml`가 Standard Generalized Markup Language (SGML)의 약자입니다. SGML은 HTML등의 Markup Language가 따라야 하는 기준을 제시하는 메타 언어입니다. 

그러므로 해당 `conll`변수는 파이썬에서 HTML내의 데이터를 추출할 때 사용하는 `BeautifulSoup` 라이브러리를 활용해 처리해보겠습니다. 아래 코드처럼 Markup Language 형태의 정보가 저장된 `conll`변수를 첫번째 파라미터로 넘기고, 두번째 파라미터에는 데이터 처리시 사용할 `parser`를 명시합니다. 일반적으로 `BeautifulSoup`에서 제공하는 `html.parser`를 사용하기도 하지만, [공식 문서](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#installing-a-parser)에서는 속도를위해 `lxml`사용하는 것을 추천하므로, 본 튜토리얼에서도 `lxml`을 활용해보겠습니다. 

In [ ]:
# 파싱하고
conllsoup = BeautifulSoup(conll, 'lxml')

In [ ]:
type(conllsoup)

bs4.BeautifulSoup

`BeautifulSoup`객체로 변환된것을 확인할 수 있습니다. 가장 먼저 CoNLL 데이터셋 내에 어떤 태그들이 존재하는지 확인해보겠습니다. 

In [ ]:
tag_names = set([tag.name for tag in conllsoup.find_all()])

List Comprehension을 활용해 `conllsoup`내에 있는 모든 태그들을 `list` 내부에 저장합니다. 그 중에는 중복되는 태그명들도 있을 수 있으므로, 고유의 태그명만 확인하기 위해 `set()`함수를 사용해 `list`를 `set` 형태로 변환시켜 줍니다. 파이썬에서 `set`은 중복되는 원소를 가질 수 없기 때문에, 중복 원소가 있는 경우 하나의 원소만 남겨두고 나머지는 제거하게 됩니다. 

`tag_names`의 있는 고유 태그들을 확인해보면 아래와 같습니다.

In [ ]:
tag_names

{'annotation',
 'body',
 'comment',
 'correction',
 'doc',
 'html',
 'mistake',
 'p',
 'text',
 'title',
 'type'}

In [ ]:
conllsoup.find('doc')

<doc nid="1">
<text>
<title>
Keeping the Secret of Genetic Testing
</title>
<p>
What is genetic risk? Genetic risk refers more to your chance of inheriting a disorder or disease. People get certain disease because of genetic changes. How much a genetic change tells us about your chance of developing a disorder is not always clear. If your genetic results indicate that you have gene changes associated with an increased risk of heart disease, it does not mean that you definitely will develop heart disease. The opposite is also true. If your genetic results show that you do not have changes associated with an increased risk of heart disease, it is still possible that you develop heart disease. However for some rare diseases, people who have certain gene changes are guaranteed to develop the disease. When we are diagonosed out with certain genetic disease, are we suppose to disclose this result to our relatives? My answer is no.
</p>
<p>
On one hand, we do not want this potential danger ca

각각의 태그와 태그 내의 속성이 의미하는 바는 다음과 같습니다. 

- `annotation`: `text`의 모든 문법 오류를 저장한 태그, `mistake`의 부모 태그
    - `teacher_id`: 문법 교정한 선생님 고유 번호
- `body`: 모든 태그 정보가 저장된 부모 태그
- `comment`: 
- `correction`: 
- `doc`: `text`, `title`, `annotation`정보가 저장된 부모 태그
    - `nid`: 문서 고유 번호
- `html`: `body`의 부모 태그
- `mistake`: `type`, `comment`, `correction` 정보가 저장된 부모 태그
    - `end_off`: 오류가 끝나는 문자열 위치
    - `end_par`: 오류가 끝나는 문단 위치
    - `start_off`: 오류가 시작하는 문자열 위치
    - `start_par`: 오류가 끝나는 문단 위치
- `p`: 에세이의 각 문단이 저장된 태그
- `text`: 학생이 쓴 에세이가 저장된 태그, `title`과 `p`의 부모 태그
- `title`: 에세이의 제목이 저장된 태그
- `type`: 문법 오류 유형이 저장된 태그


해당 에세이를 독해 문제 지문으로 활용하기 위해선 탐지된 문법 오류를 모두 교정할 필요가 있습니다. 문법 오류가 발생한 위치 정보와 해당 단어를 대체하기 위한 올바른 문법 단어 정보를 알고 있으므로, 적절한 알고리즘을 활용해 에세이내의 모든 문법 오류를 대체할 수 있습니다. 해당 과정을 2.1.2절에서 확인해보겠습니다. 

### 2.1.2 CoNLL 데이터 전처리

In [ ]:
sample = 'i can do this all day'

In [ ]:
sample[2:5] = 'will'

TypeError: ignored

In [ ]:
sample[:2] + 'will' + sample[5:]

'i will do this all day'

In [ ]:
import numpy as np

def edit_paragraph(paragraph, start, end, correction):
    return paragraph[:start] + correction + paragraph[end:]

def update_index(paragraph, offset, corrections):
    '''
    paragraph: 원문
    offset: 원본 위치정보
    corrections: 수정해야 하는 단어
    '''
    raw_words = []
    # 1. detect all the words at offset values in the paragraph / O(offset)
    for of in offset:
        raw_words.append(paragraph[of['start_off']:of['end_off']])

    # 2. compare length change between the detected words and corrections (map is faster than list comprehension when it doesn't use lambda) / O(offset)
    adjusted = np.array(list(map(len, corrections))) - np.array(list(map(len, raw_words)))

    # cumulate adjusted values / O(offset)
    for i in range(1, len(adjusted)):
        adjusted[i] += adjusted[i-1]
    
    # 3. adjust the index sequentially / O(offset)
    for i in range(1, len(offset)):
        offset[i]['start_off'] += adjusted[i-1]
        offset[i]['end_off'] += adjusted[i-1]

In [ ]:
all_docs = revisedsoup.find_all('doc')

In [ ]:
# strip은 앞뒤 공란 제거
_t = all_docs[0].find('text').text.strip().split('\n\n\n')

In [ ]:
_t

['Keeping the Secret of Genetic Testing',
 'What is genetic risk? Genetic risk refers more to your chance of inheriting a disorder or disease. People get certain disease because of genetic changes. How much a genetic change tells us about your chance of developing a disorder is not always clear. If your genetic results indicate that you have gene changes associated with an increased risk of heart disease, it does not mean that you definitely will develop heart disease. The opposite is also true. If your genetic results show that you do not have changes associated with an increased risk of heart disease, it is still possible that you develop heart disease. However for some rare diseases, people who have certain gene changes are guaranteed to develop the disease. When we are diagonosed out with certain genetic disease, are we suppose to disclose this result to our relatives? My answer is no.',
 "On one hand, we do not want this potential danger causing firghtenning affects in our familie

In [ ]:
all_docs = conllsoup.find_all('doc')
corrected_essays = []

for doc in all_docs:
    
    # extract paragraphs
    paragraph_lists = doc.find('text').text.strip().split('\n\n\n')

    # paragraph 별로 수정 진행
    # make list that contains offset values

    all_mistakes = doc.find_all('mistake')

    for para_idx, para in enumerate(paragraph_lists):
        edits = [mistake.attrs for mistake in all_mistakes if mistake.attrs['start_par'] == f'{para_idx}']

        if len(edits) == 0:
            continue
        
        edits = [{key: int(value) for key, value in edit.items()} for edit in edits]

        # make list that contains correction information 
        corrections = [mistake.find('correction').text for mistake in all_mistakes if mistake.attrs['start_par'] == f'{para_idx}'] #start par과 end par은 항상 같은가?
        
        # reindexing
        update_index(para, edits, corrections)

        for edit_idx, edit in enumerate(edits):
            para = edit_paragraph(para, int(edit['start_off']), int(edit['end_off']), corrections[edit_idx])

        paragraph_lists[para_idx] = para

    corrected_essays.append(' '.join(paragraph_lists))

In [ ]:
len(corrected_essays)

50

In [ ]:
import pickle

In [ ]:
sample_list = [1, 2, 3]
file_name = "corrected_essays.pkl"

open_file = open(file_name, "wb")
pickle.dump(corrected_essays, open_file)
open_file.close()



In [ ]:
import pickle
file_name = "corrected_essays.pkl"
open_file = open(file_name, "rb")
loaded_list = pickle.load(open_file)
open_file.close()

In [ ]:
loaded_list[0]

"Keeping the Secret of Genetic Testing What is genetic risk? Genetic risk refers  to your chance of inheriting a disorder or disease. People get certain diseases because of genetic changes. How much a genetic change tells us about your chance of developing a disorder is not always clear. If your genetic results indicate that you have gene changes associated with an increased risk of heart disease, it does not mean that you definitely will develop heart disease. The opposite is also true. If your genetic results show that you do not have changes associated with an increased risk of heart disease, it is still possible that you will develop heart disease. However, for some rare diseases, people who have certain gene changes are guaranteed to develop the disease. When we are diagnosed with a certain genetic disease, are we supposed to disclose this result to our relatives? My answer is no. On the one hand, we do not want this potential danger to have frightening effects on our families' la

In [ ]:
corrected_essays[0]

'Keeping the Secret of Genetic Testing What is genetic risk? Genetic risk refers  to your chance of inheriting a disorder or disease. People get certain diseases because of genetic changes. How much a genetic change tells us about your chance of developing a disorder is not always clear. If your genetic results indicate that you have gene changes associated with an increased risk of heart disease, it does not mean that you definitely will develop heart disease. The opposite is also true. If your genetic results show that you do not have changes associated with an increased risk of heart disease, it is still possible that you develop heart disease. However, for some rare diseases, people who have certain gene changes are guaranteed to develop the disease. When we are diagnosed with certain genetic diseases, are we suppose to disclose this result to our relatives? My answer is no. On one hand, we do not want this potential danger havingfrightening effects in our families\' later lives. W

## 2.2 2019 BEA-2019 Shared Task Dataset

BEA Workshop은 Association for Computational Linguistics special interest group for building educational applications (SIGEDU)에서 매년 운영하는 워크숍입니다. BEA Workshop은 Workshop on Innovative Use of NLP for Building Educational Applications의 약자이며, NLP를 교육 도메인에 적용하는 활용 사례들을 다루는 워크숍입니다. 

2019년에 개최된 BEA-2019 워크숍에서 Shared Task로 GEC 문제를 다뤘습니다. 해당 대회에서 새롭게 제공한 데이터셋은 Cambridge English Write & Improve (W&I) 데이터와 LOCNESS corpus 입니다.

Write & Improve는 비영어권 학생들이 글쓰기 자료를 올려서 향상할 수 있는 방법을 피드백 받는 온라인 플랫폼입니다. 2014년도 부터 W&I 데이터 어노테이터 (Annotator)들이 제출물들의 문법 오류를 수기로 기록을 해왔기 때문에, 본 대회에서는 해당 데이터를 훈련용, 검증용, 시험용 데이터로 공개했습니다. 

그 외에도 LOCNESS corpus를 공개했습니다. LOCNESS는 영국과 미국의 원어민 학부생들이 작성한 약 400개의 에세이가 담긴 말뭉치입니다. 필터링 과정을 거친 총 100개의 에세이를 W&I 데이터 어노테이터들이 라벨링 해서 검증용, 시험용 데이터로 공개했습니다. 샘플 개수가 100개 밖에 되지 않아 별도로 훈련용 데이터로는 제공하지 않았습니다. 





- Creator: University of Cambridge
- Year: 2019
- Purpose: 
    - 
- Data:
    - represents a wider range of native and learner
English levels and abilities compared to CoNLL
    -  350 essays (4,477 sentences) on approximately 50 topics written by 334 authors from around the world (including native English speakers)

- Papers: https://www.aclweb.org/anthology/W19-4406.pdf

### 2.2.1 W&I + LOCNESS 데이터 탐색

In [ ]:
!wget -O 'bea19.tar.gz' https://www.cl.cam.ac.uk/research/nl/bea2019st/data/wi+locness_v2.1.bea19.tar.gz

--2021-04-04 18:45:41--  https://www.cl.cam.ac.uk/research/nl/bea2019st/data/wi+locness_v2.1.bea19.tar.gz
Resolving www.cl.cam.ac.uk (www.cl.cam.ac.uk)... 128.232.0.20, 2a05:b400:110::80:14
Connecting to www.cl.cam.ac.uk (www.cl.cam.ac.uk)|128.232.0.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6120469 (5.8M) [application/x-gzip]
Saving to: ‘bea19.tar.gz’

bea19.tar.gz        100%[===================>]   5.84M  5.09MB/s    in 1.1s    

2021-04-04 18:45:43 (5.09 MB/s) - ‘bea19.tar.gz’ saved [6120469/6120469]



In [ ]:
!tar -xf bea19.tar.gz

In [ ]:
path = 'wi+locness/json/C.dev.json'

In [ ]:
with open(path) as f:
  data = f.readlines()

In [ ]:
data[0]

'{"edits": [[0, [[89, 98, "saw"], [316, 316, " for"], [343, 343, " for"], [365, 365, " a"], [375, 375, " the"], [406, 406, " a"], [469, 471, "of"], [523, 527, "know"], [589, 594, "chips"]]]], "userid": "24796", "text": "Dear Mrs. Ashby, \\n\\nYesterday I was in Green Pepper Cafe for a meal with colleagues and I have seen the advertisement for a job at weekends in your cafe. \\n\\nI am very interested in this work and believe that my employment background is appropriate for it. \\n\\nI am a Tourism student and I need to work at weekends to pay my studies. \\nI have worked one year in London as waiter in Hard Rock Cafe and 6 months as waiter also in Barcomi\\u2019s Cafe in Berlin. So I have experience in service, costumer care and working long hours. \\n\\nI Know how to prepare different kinds of food: sandwiches, fish and fries, hamburgers, Italian pasta, etc \\n\\nI am also very good at dealing with people, I have never had a complaint sheet!\\n\\nI have total availability at weekends 

In [ ]:
import json

data = list(map(json.loads, data))

In [ ]:
len(data)

70

In [ ]:
data[2]['text'][2766:2777]

'her despite'

In [ ]:
# [0][1] used to parse the location of edits and corrections
data[2]['edits'][0][1]

[[154, 154, ' any'],
 [155, 164, 'occurrence'],
 [254, 254, ','],
 [414, 414, ','],
 [490, 490, ','],
 [576, 576, ','],
 [672, 672, ','],
 [682, 690, '. Because'],
 [706, 706, ','],
 [798, 809, "sweetheart's"],
 [830, 834, 'shows'],
 [988, 989, ';'],
 [1112, 1115, ''],
 [1284, 1284, ','],
 [1655, 1655, ','],
 [1807, 1807, ','],
 [1849, 1853, 'led'],
 [1859, 1862, 'from'],
 [1877, 1877, ','],
 [1902, 1902, ','],
 [2373, 2380, 'leaves -'],
 [2442, 2445, 'that'],
 [2511, 2511, ','],
 [2598, 2602, 'grows'],
 [2766, 2777, 'he. Despite'],
 [2830, 2830, ','],
 [2989, 2989, ',']]

### 2.2.2 W&I + LOCNESS 데이터 전처리

In [ ]:
import numpy as np

def edit_paragraph(paragraph, start, end, correction):
    return paragraph[:start] + correction + paragraph[end:]

def update_index_BEA(paragraph, edits):
    '''
    paragraph: 원문
    edits: edits from BEA_2019
    '''
    raw_words = []
    corrections = []

    # 1. detect all the words at offset values in the paragraph / O(offset)
    for edit in edits:
        raw_words.append(paragraph[edit[0]:edit[1]])
        # make corrections
        corrections.append(edit[2])

    # 2. compare length change between the detected words and corrections (map is faster than list comprehension when it doesn't use lambda) / O(offset)
    adjusted = np.array(list(map(len, corrections))) - np.array(list(map(len, raw_words)))

    # cumulate adjusted values / O(offset)
    for i in range(1, len(adjusted)):
        adjusted[i] += adjusted[i-1]
    
    # 3. adjust the index sequentially / O(offset)
    for i in range(1, len(edits)):
        edits[i][0] += adjusted[i-1]
        edits[i][1] += adjusted[i-1]

    return corrections

In [ ]:
raw_words = []
corrections = []
for edit in edits:
    raw_words.append(data[9]['text'][edit[0]:edit[1]])
    corrections.append(edit[2])

In [ ]:
data[9]['text']

"On Britains roads there is an ever increasing volume of traffic. The road network is no longer able to carry this traffic without hold-ups and traffic jams.. To remedy this the government has started adding a fourth lane on some streches of our motorways and constructing ring roads and bypasses, with mixed reception. While the road system is being improoved there seems to be an endless series of roadworks which restrict the highways and cause further delay while the roadworks are in place. The inability to cope with the ammount of traffic by the road system obviously increases the risk of drives having an accident and the drivers have to be constantly alert as they are nearly always in capacity traffic. It might seem an easy soloution to this  mayhem would be to use public transport; i.e. the Railways. People are not taking to the Rail system because of its lack of integration due to the recent privitisation of different areas. It has been shown that different companies charge differe

In [ ]:
data[9]['text'][1422:1438]

'look forwards to'

In [ ]:
corrections

["Britain's",
 '-',
 '',
 ',',
 'stretches',
 'improved,',
 'delays',
 'amount',
 'on the part of',
 'drivers',
 None,
 'solution',
 'railways',
 'rail',
 'privatisation',
 ',',
 ',',
 ',',
 'timetable',
 ',',
 None,
 'bus',
 ' pay for',
 'solution',
 '. This',
 'off',
 'improve',
 ',',
 ',',
 'liaise',
 ',',
 ' as',
 'than']

In [ ]:
[edit for edit in edits if edit[2] != None]

[[3, 11, "Britain's"],
 [34, 35, '-'],
 [156, 157, ''],
 [172, 172, ','],
 [229, 237, 'stretches'],
 [350, 359, 'improved,'],
 [455, 460, 'delays'],
 [526, 533, 'amount'],
 [545, 547, 'on the part of'],
 [596, 602, 'drivers'],
 [735, 744, 'solution'],
 [804, 812, 'railways'],
 [843, 847, 'rail'],
 [908, 921, 'privatisation'],
 [1078, 1078, ','],
 [1160, 1160, ','],
 [1265, 1265, ','],
 [1287, 1297, 'timetable'],
 [1395, 1395, ','],
 [1463, 1466, 'bus'],
 [1469, 1469, ' pay for'],
 [1497, 1506, 'solution'],
 [1586, 1592, '. This'],
 [1610, 1612, 'off'],
 [1647, 1655, 'improve'],
 [1672, 1672, ','],
 [1734, 1734, ','],
 [1828, 1833, 'liaise'],
 [1901, 1901, ','],
 [1921, 1921, ' as'],
 [1930, 1932, 'than']]

In [ ]:
edits = data[9]['edits'][0][1]

corrections = update_index_BEA(data[9]['text'], edits)

edits

TypeError: ignored

In [ ]:
edits

[[3, 11, "Britain's"],
 [34, 35, '-'],
 [156, 157, ''],
 [172, 172, ','],
 [229, 237, 'stretches'],
 [350, 359, 'improved,'],
 [455, 460, 'delays'],
 [526, 533, 'amount'],
 [545, 547, 'on the part of'],
 [596, 602, 'drivers'],
 [695, 703, None],
 [735, 744, 'solution'],
 [804, 812, 'railways'],
 [843, 847, 'rail'],
 [908, 921, 'privatisation'],
 [1078, 1078, ','],
 [1160, 1160, ','],
 [1265, 1265, ','],
 [1287, 1297, 'timetable'],
 [1395, 1395, ','],
 [1422, 1438, None],
 [1463, 1466, 'bus'],
 [1469, 1469, ' pay for'],
 [1497, 1506, 'solution'],
 [1586, 1592, '. This'],
 [1610, 1612, 'off'],
 [1647, 1655, 'improve'],
 [1672, 1672, ','],
 [1734, 1734, ','],
 [1828, 1833, 'liaise'],
 [1901, 1901, ','],
 [1921, 1921, ' as'],
 [1930, 1932, 'than']]

In [ ]:
para = data[2]['text']

for edit_idx, edit in enumerate(edits):
    para = edit_paragraph(para, edit[0], edit[1], corrections[edit_idx])


In [ ]:
data[2]['text']

"How does Voltaire tackle the question of philosophical optimism in 'Candide'?\n\n\tPhilosophical optimism -l'optimisme- is the philosophy that everything and occurence is for some good. Voltaire sets out in Candide to criticise this philosophy.\n\n\tIn Candide the representative of this philosophy is Pangloss, the philosopher employed at Thunder-der-Troncks.\n\n\tThe whole book is a criticism of l'optimisme. For example when Candide is forced to join the Bulgar army he is beaten almost to death but Candide does not concern himself because he know it is for his own good - in fact later in the book he uses his training to become a soldier. Although this would seem to be good it is not because of his training he kills two men, a Jew and an Archbishop with no hesitation and then proceeds to kill his sweethearts brother.\n\n\tVoltaire show his dislike of l'optimisme when Pangloss, Candide and James are caught in a bad storm at sea in which James is drowned and there are only three survivor

In [ ]:
' '.join(para.split('\n\n\t'))

"How does Voltaire tackle the question of philosophical optimism in 'Candide'? Philosophical optimism -l'optimisme- is the philosophy that everything and any occurrence is for some good. Voltaire sets out in Candide to criticise this philosophy. In Candide, the representative of this philosophy is Pangloss, the philosopher employed at Thunder-der-Troncks. The whole book is a criticism of l'optimisme. For example, when Candide is forced to join the Bulgar army he is beaten almost to death, but Candide does not concern himself because he know it is for his own good - in fact, later in the book he uses his training to become a soldier. Although this would seem to be good, it is not. Because of his training, he kills two men, a Jew and an Archbishop with no hesitation and then proceeds to kill his sweetheart's brother. Voltaire shows his dislike of l'optimisme when Pangloss, Candide and James are caught in a bad storm at sea in which James is drowned and there are only three survivors; Can

In [ ]:
len(data)

50

In [ ]:
data[0]['text']

'Boxing is a common, well known and well loved sport amongst most countries in the world however it is also punishing, dangerous and disliked to the extent that many people want it banned, possibly with good reason.\nBoxing is a dangerous sport, there are relatively common deaths, tragic injuries and even disease. All professional boxers are at risk from being killed in his next fight. If not killed then more likely paralysed. There have been a number of cases in the last ten years of the top few boxers having tragic losses throughout their ranks. This is just from the elite few, and theres more from those below them.\nMore deaths would occur through boxing if it were banned. The sport would go underground, there would be no safety measures like gloves, a doctor, paramedics or early stopping of the fight if someone looked unable to continue. With this going on the people taking part will be dangerous, and on the streets. Dangerous dogs who were trained to kill and maim in similar under

In [ ]:
from tqdm import tqdm

In [ ]:
corrected_essays = []

for doc in tqdm(data):
    
    # extract paragraphs
    essay = doc['text']

    # paragraph 별로 수정 진행
    # make list that contains offset values

    edits = doc['edits'][0][1]
    edits = [edit for edit in edits if edit[2] != None]
    corrections = update_index_BEA(essay, edits)    

    for edit_idx, edit in enumerate(edits):
        essay = edit_paragraph(essay, edit[0], edit[1], corrections[edit_idx])

    corrected_essays.append(' '.join(essay.split('\n\n\t')))

In [ ]:
data[49]['text']

'Violence on television can be seen throughout the many channels that cable has to offer.  With an average national TV viewing time of 7 1/4 hours daily, the prevalence of violence in broadcasts is a serious concern.  Television programming in the United States is considered the most violent in advanced industrialized nations (Hepburn 1).  Many children watch violent tv shows because the action thrills them and can sometimes cause the child to become violent.  Violence is common in TV entertainment-- the dramas that portray stories about crime, psychotic murderers, police cases, emergency services, international terrorism, and war (Hepburn 1).  Television violence is prevalent in the United States and some people disagree with its effects on children.\nThe V-chip is an electrical device that blocks out violent television shows.  Some people, such as the Senate Commerce Committee, are not satisfied with it.  The Senate Commerce Committee approved bills creating a partial ban on TV viole

In [ ]:
corrected_essays[49]

'Violence on television can be seen throughout the many channels that cable has to offer.  With an average national TV viewing time of 7 1/4 hours daily, the prevalence of violence in broadcasts is a serious concern.  Television programming in the United States is considered the most violent in advanced industrialized nations (Hepburn 1).  Many children watch violent TV shows because the action thrills them and can sometimes cause the child to become violent.  Violence is common in TV entertainment-- the dramas that portray stories about crime, psychotic murderers, police cases, emergency services, international terrorism, and war (Hepburn 1).  Television violence is prevalent in the United States and some people disagree about its effects on children.\nThe V-chip is an electrical device that blocks out violent television shows.  Some people, such as the Senate Commerce Committee, are not satisfied with it.  The Senate Commerce Committee approved bills creating a partial ban on TV viol

In [ ]:
corrected_essays

[]

In [ ]:
corrected_essays

["Boxing is a common, well-known and well-loved sport in most co. However,in the world however it is also punishing, dangerous and disliked to the extent that many people want it banned, possibly with good reason.\nBoxing is a dan. Theresport, there are relatively common deaths, tragic injuries and even disease. All professional boxers are at risk from beintheirilled in his next fight. If not ki,lled then more likely paralysed. There have been a number of cases in the last ten years of the top few boxers having tragic losses among their ranks. This is just fromthere'slite few, and theres more from those below them.\nMore deaths would occur through boxing if it were banned. The sport would and go underground, there would be no safety measures like gloves, a doctor, paramedics or early stopping of the fight if someone looked unable to continue. With this going on, the people taking part will be dangerous, and on the streets. Dangerous dogs who were trained to kill and maim in similar und

In [ ]:
type(t)

_io.TextIOWrapper

In [ ]:
def squad_json_to_dataframe_train(input_file_path, record_path = ['data','paragraphs','qas','answers'],
                           verbose = 1):
    """
    input_file_path: path to the squad json file.
    record_path: path to deepest level in json file default value is
    ['data','paragraphs','qas','answers']
    verbose: 0 to suppress it default is 1
    """
    if verbose:
        print("Reading the json file")    
    file = json.loads(open(input_file_path).read())
    if verbose:
        print("processing...")
    # parsing different level's in the json file
    js = pd.io.json.json_normalize(file , record_path )
    m = pd.io.json.json_normalize(file, record_path[:-1] )
    r = pd.io.json.json_normalize(file,record_path[:-2])
    
    #combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
    ndx  = np.repeat(m['id'].values,m['answers'].str.len())
    m['context'] = idx
    js['q_idx'] = ndx
    main = pd.concat([ m[['id','question','context']].set_index('id'),js.set_index('q_idx')],1,sort=False).reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape))
        print("Done")
    return main

## 참고문헌

[1] Ng, H. T., Wu, S. M., Briscoe, T., Hadiwinoto, C., Susanto, R. H., & Bryant, C. (2014). The CoNLL-2014 Shared Task on Grammatical Error Correction. Proceedings of the Eighteenth Conference on Computational Natural Language Learning: Shared Task, 1–14. https://doi.org/10.3115/v1/W14-1701

[2] Bryant, C., Felice, M., Andersen, I. E., & Briscoe, T. (2019). The BEA-2019 Shared Task on Grammatical Error Correction. Proceedings of the Fourteenth Workshop on Innovative Use of NLP for Building Educational Applications, 52–75. https://doi.org/10.18653/v1/w19-4406

[3] https://www.differencebetween.com/difference-between-html-and-vs-sgml/




In [ ]:
["essay1", "essay2", "essay3"]

In [ ]:
ch6. reference 

In [ ]:
!pip install benepar==0.1.2
!pip install summa
!pip install nltk==3.4.5
!pip install spacy==2.1.0
!python3 -m spacy download en
!pip install scipy

In [ ]:
!pip install tensorflow==1.14.0 


     |████████████████████████████████| 109.3MB 94kB/s 
     |████████████████████████████████| 3.2MB 44.1MB/s 
     |████████████████████████████████| 491kB 48.4MB/s 
     |████████████████████████████████| 51kB 4.6MB/s 
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


In [ ]:
import requests
import json
from summa.summarizer import summarize
import benepar
import string
import nltk
from nltk import tokenize
from nltk.tokenize import sent_tokenize
import re
from random import shuffle
import spacy
nlp = spacy.load('en')
#this package is required for the summa summarizer
nltk.download('punkt')
benepar.download('benepar_en2')
benepar_parser = benepar.Parser("benepar_en2")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
sentences[0]

In [ ]:
from string import punctuation

def preprocess(sentences):
    output = []
    for sent in sentences:
        single_quotes_present = len(re.findall(r"['][\w\s.:;,!?\\-]+[']",sent))>0
        double_quotes_present = len(re.findall(r'["][\w\s.:;,!?\\-]+["]',sent))>0
        question_present = "?" in sent
        if single_quotes_present or double_quotes_present or question_present :
            continue
        else:
            output.append(sent.strip(punctuation))
    return output
        
        
def get_candidate_sents(resolved_text, ratio=0.3):
    candidate_sents = summarize(resolved_text, ratio=ratio)
    candidate_sents_list = tokenize.sent_tokenize(candidate_sents)
    candidate_sents_list = [re.split(r'[:;]+',x)[0] for x in candidate_sents_list ]
    # Remove very short sentences less than 30 characters and long sentences greater than 150 characters
    filtered_list_short_sentences = [sent for sent in candidate_sents_list if len(sent)>30 and len(sent)<150]
    return filtered_list_short_sentences

cand_sents = get_candidate_sents(sentences[0])
filter_quotes_and_questions = preprocess(cand_sents)
for each_sentence in filter_quotes_and_questions:
    print (each_sentence)
    print ("\n")

The convenience and high efficiency of using electronic products is being noticed by people worldwide


In this essay, I will discuss that if surveillance technology such as RFID (radio-frequency identification) should be used or not to track people




In [ ]:

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
    
def get_flattened(t):
    sent_str_final = None
    if t is not None:
        sent_str = [" ".join(x.leaves()) for x in list(t)]
        sent_str_final = [" ".join(sent_str)]
        sent_str_final = sent_str_final[0]
    return sent_str_final
    

def get_termination_portion(main_string,sub_string):
    combined_sub_string = sub_string.replace(" ","")
    main_string_list = main_string.split()
    last_index = len(main_string_list)
    for i in range(last_index):
        check_string_list = main_string_list[i:]
        check_string = "".join(check_string_list)
        check_string = check_string.replace(" ","")
        if check_string == combined_sub_string:
            return " ".join(main_string_list[:i])
                     
    return None
    
def get_right_most_VP_or_NP(parse_tree,last_NP = None,last_VP = None):
    if len(parse_tree.leaves()) == 1:
        return get_flattened(last_NP),get_flattened(last_VP)
    last_subtree = parse_tree[-1]
    if last_subtree.label() == "NP":
        last_NP = last_subtree
    elif last_subtree.label() == "VP":
        last_VP = last_subtree
    
    return get_right_most_VP_or_NP(last_subtree,last_NP,last_VP)


def get_sentence_completions(key_sentences):
    sentence_completion_dict = {}
    for individual_sentence in filter_quotes_and_questions:
        sentence = individual_sentence.rstrip('?:!.,;')
        tree = benepar_parser.parse(sentence)
        last_nounphrase, last_verbphrase =  get_right_most_VP_or_NP(tree)
        phrases= []
        if last_verbphrase is not None:
            verbphrase_string = get_termination_portion(sentence,last_verbphrase)
            phrases.append(verbphrase_string)
        if last_nounphrase is not None:
            nounphrase_string = get_termination_portion(sentence,last_nounphrase)
            phrases.append(nounphrase_string)

        longest_phrase =  sorted(phrases, key=len,reverse= True)
        if len(longest_phrase) == 2:
            first_sent_len = len(longest_phrase[0].split())
            second_sentence_len = len(longest_phrase[1].split())
            if (first_sent_len - second_sentence_len) > 4:
                del longest_phrase[1]
                
        if len(longest_phrase)>0:
            sentence_completion_dict[sentence]=longest_phrase
    return sentence_completion_dict



sent_completion_dict = get_sentence_completions(filter_quotes_and_questions)

print (sent_completion_dict)


    

{'The convenience and high efficiency of using electronic products is being noticed by people worldwide': ['The convenience and high efficiency of using electronic products is being noticed by', 'The convenience and high efficiency of using electronic products is being'], 'In this essay, I will discuss that if surveillance technology such as RFID (radio-frequency identification) should be used or not to track people': ['In this essay, I will discuss that if surveillance technology such as RFID (radio-frequency identification) should be used or not to track', 'In this essay, I will discuss that if surveillance technology such as RFID (radio-frequency identification) should be used or not to']}


In [ ]:
sent_completion_dict['The convenience and high efficiency of using electronic products is being noticed by people worldwide']

['The convenience and high efficiency of using electronic products is being noticed by',
 'The convenience and high efficiency of using electronic products is being']

In [ ]:
sent_completion_dict['In this essay, I will discuss that if surveillance technology such as RFID (radio-frequency identification) should be used or not to track people']

['In this essay, I will discuss that if surveillance technology such as RFID (radio-frequency identification) should be used or not to track',
 'In this essay, I will discuss that if surveillance technology such as RFID (radio-frequency identification) should be used or not to']

In [ ]:

!pip install sentence-transformers==0.2.5.1
!pip install transformers==2.6.0

In [ ]:

from transformers import GPT2LMHeadModel, GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
model = GPT2LMHeadModel.from_pretrained("gpt2",pad_token_id=tokenizer.eos_token_id)

In [ ]:
from sentence_transformers import SentenceTransformer
# Load the BERT model. Various models trained on Natural Language Inference (NLI) https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/nli-models.md and 
# Semantic Textual Similarity are available https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/sts-models.md
model_BERT = SentenceTransformer('bert-base-nli-mean-tokens')

100%|██████████| 405M/405M [00:32<00:00, 12.5MB/s]


In [ ]:
import torch

In [ ]:
from nltk import tokenize
import scipy
torch.manual_seed(2020)


def sort_by_similarity(original_sentence,generated_sentences_list):
    # Each sentence is encoded as a 1-D vector with 768 columns
    sentence_embeddings = model_BERT.encode(generated_sentences_list)

    queries = [original_sentence]
    query_embeddings = model_BERT.encode(queries)
    # Find the top sentences of the corpus for each query sentence based on cosine similarity
    number_top_matches = len(generated_sentences_list)

    dissimilar_sentences = []

    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]

        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])


        for idx, distance in reversed(results[0:number_top_matches]):
            score = 1-distance
            if score < 0.9:
                dissimilar_sentences.append(generated_sentences_list[idx].strip())
           
    sorted_dissimilar_sentences = sorted(dissimilar_sentences, key=len)
    
    return sorted_dissimilar_sentences[:3]
    

def generate_sentences(partial_sentence,full_sentence):
    input_ids = torch.tensor([tokenizer.encode(partial_sentence)])
    maximum_length = len(partial_sentence.split())+80

    # Actiavte top_k sampling and top_p sampling with only from 90% most likely words
    sample_outputs = model.generate(
        input_ids, 
        do_sample=True, 
        max_length=maximum_length, 
        top_p=0.90, # 0.85 
        top_k=50,   #0.30
        repetition_penalty  = 10.0,
        num_return_sequences=10
    )
    generated_sentences=[]
    for i, sample_output in enumerate(sample_outputs):
        decoded_sentences = tokenizer.decode(sample_output, skip_special_tokens=True)
        decoded_sentences_list = tokenize.sent_tokenize(decoded_sentences)
        generated_sentences.append(decoded_sentences_list[0])
        
    top_3_sentences = sort_by_similarity(full_sentence,generated_sentences)
    
    return top_3_sentences

index = 1
choice_list = ["a)","b)","c)","d)","e)","f)"]
for key_sentence in sent_completion_dict:
    partial_sentences = sent_completion_dict[key_sentence]
    false_sentences =[]
    print_string = "**%s) True Sentence (from the story) :**"%(str(index))
    printmd(print_string)
    print ("  ",key_sentence)
    for partial_sent in partial_sentences:
        false_sents = generate_sentences(partial_sent,key_sentence)
        false_sentences.extend(false_sents)
    printmd("  **False Sentences (GPT-2 Generated)**")
    for ind,false_sent in enumerate(false_sentences):
        print_string_choices = "**%s** %s"%(choice_list[ind],false_sent)
        printmd(print_string_choices)
    index = index+1
    
    print ("\n\n")
        



**1) True Sentence (from the story) :**

   The convenience and high efficiency of using electronic products is being noticed by people worldwide


  **False Sentences (GPT-2 Generated)**

**a)** The convenience and high efficiency of using electronic products is being noticed by more than 20 million consumers.

**b)** The convenience and high efficiency of using electronic products is being noticed by the general public, but this has not been recognized at a rapid pace.

...

**b)** In this essay, I will discuss that if surveillance technology such as RFID (radio-frequency identification) should be used or not to track individuals who do something in the privacy of their own homes instead rather than at work.

**c)** In this essay, I will discuss that if surveillance technology such as RFID (radio-frequency identification) should be used or not to track every location of someone who has paid the bill and is suspected by law enforcement agencies.

**d)** In this essay, I will discuss that if surveillance technology such as RFID (radio-frequency identification) should be used or not to keep track of what is going on inside our homes and businesses.

**e)** In this essay, I will discuss that if surveillance technology such as RFID (radio-frequency identification) should be used or not to identify individuals from the United States and other places then there are good reasons for using it.

**f)** In this essay, I will discuss that if surveillance technology such as RFID (radio-frequency identification) should be used or not to locate terrorist suspects using search warrants and/or law enforcement agencies it is only because of a need for effective monitoring.

In [ ]:
sentences[2]

'\nRisks can be analyzed if there is a record of what was happened in the past. In late nineteenth century, there was a severe air crash happening at Miami international airport. The air cargo of the Valujet plane was on fire after the plane had taken off. Sooner or later, the electric systems were short circuited as a result that the airplane was out of control and nose diving to the ground with extremely high speed. All passengers and pilots were died. After finding and analyzing the black box of the Valujet after the incidence, police investigators found out reasons for causing this severe incidence. Surveillance technology can be used to detect very risky events and prevent its happening in the future, hence to save more lives. In addition, if surveillance technology is even better developed, it can be used to detect the problem before the real accidence has happened.\n'